<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Errors-Analysis" data-toc-modified-id="Errors-Analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Errors Analysis</a></span><ul class="toc-item"><li><span><a href="#1.-Kappa" data-toc-modified-id="1.-Kappa-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. Kappa</a></span></li><li><span><a href="#2.-Most-common-erros-in-POS" data-toc-modified-id="2.-Most-common-erros-in-POS-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2. Most common erros in POS</a></span></li><li><span><a href="#2.-Differences-in-errors-between-1-tagger" data-toc-modified-id="2.-Differences-in-errors-between-1-tagger-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>2. Differences in errors between 1-tagger</a></span><ul class="toc-item"><li><span><a href="#2.1.-Correlation-and-Confustion-Matrix" data-toc-modified-id="2.1.-Correlation-and-Confustion-Matrix-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>2.1. Correlation and Confustion Matrix</a></span></li></ul></li></ul></li><li><span><a href="#Convert-To-Conllu" data-toc-modified-id="Convert-To-Conllu-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Convert To Conllu</a></span></li></ul></div>

In [ ]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import itertools

In [ ]:
from os import listdir
from os.path import join, isdir, exists, isfile, getmtime
listoffiles = [f[:-5] for f in listdir(".") if f.endswith(".xlsx") and f.startswith("results_")]
listoffiles

In [ ]:
listofdates = {i: getmtime(i+".xlsx") for i in listoffiles}

In [ ]:
dfs= {x: pd.read_excel(x+'.xlsx', header=[0,1]) for x in listoffiles}

In [ ]:
try:
    accuracies
except NameError:
    accuracies = pd.DataFrame([],
                 columns=list(map(list, zip(*[
        ('miscs', 'date'),
        ('QAaspect', 'acc'),
        ('QAcase', 'acc'),
        ('QAgender', 'acc'),
        ('QAmood', 'acc'),
        ('QAnumber', 'acc'),
        ('QAperson', 'acc'),
        ('QApos', 'acc'),
        ('QAstate', 'acc'),
        ('QAvoice', 'acc'),
        ('QAutf8', 'acc'),
        ('out_diac', 'acc'),
        ('out_letter', 'acc'),
        ('seg', 'acc'),
        ('QAupos', 'acc'),
        ('agg', 'all'),
        ('agg', 'mf'),
        ('agg', 'utf8')]))))

In [ ]:
import numpy as np
import time
for f in listoffiles:
    for x in list(accuracies.columns):
        if x in dfs[f].columns:
            accuracies.loc[f+"_test", x] = np.average(dfs[f][x].values.astype(int),axis=0)
    accuracies.loc[f+"_test", ("miscs","date")] = time.ctime(listofdates[f])


In [ ]:
dfs["results_comp_end_baseline_1"]

In [ ]:
writer_all = pd.ExcelWriter('kappa.xlsx')

# Errors Analysis

## 1. Kappa 

In [ ]:
tools = ["baseline","MX", "FA", "AM", "ST"]
outputs = ['QAaspect', 'QAcase', 'QAgender', 'QAmood', 'QAnumber', 'QAperson', 'QApos', 'QAstate', 'QAutf8', 'QAvoice']
for L in range(2, 3): #len(tools)+1):
    for o in outputs:
        table = pd.DataFrame(1, columns=tools, index=tools)
        for subset in itertools.combinations(tools, L):
            table.loc[subset[0],subset[1]]= cohen_kappa_score(dfs["results_comp_end_"+subset[0]+"_1"][(o,"pred")].astype(str), dfs["results_comp_end_"+subset[1]+"_1"][(o,"pred")].astype(str))
            table.loc[subset[1],subset[0]] = table.loc[subset[0],subset[1]]
#             print(subset, o, cohen_kappa_score(dfs["results_comp_end_"+subset[0]+"_1"][(o,"pred")].astype(str), dfs["results_comp_end_"+subset[1]+"_1"][(o,"pred")].astype(str)))
        table.to_excel(writer_all,o)
        print(o)
        print(table)
writer_all.save()

## 2. Most common erros in POS

In [ ]:
res = dfs["results_comp_end_decoder_for_each"]

from difflib import SequenceMatcher

def findDiffs(string1,string2,i):
    match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))

    return (string1.replace(string1[match.a: match.a + match.size],""), 
            string2.replace(string2[match.b: match.b + match.size],""),i)


res[res["QApos","acc"]==False][["QApos","QAutf8"]]

res[res["QApos","acc"]==False].iloc[112]

from collections import Counter
def printDifferencesBetweenColumns(res,col1,col2):
    return [findDiffs(res.iloc[i][col1],res.iloc[i][col2],0) for i in range(len(res))]
Counter(printDifferencesBetweenColumns(res[res["QApos","acc"]==False],("QApos","pred"),("QApos","actu"))).most_common()

## 2. Differences in errors between 1-tagger

### 2.1. Correlation and Confustion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
for t in dfs.keys():
    for t2 in dfs.keys():
        if t == t2:
            continue
        merged = pd.merge(dfs[t],dfs[t2], left_index=True, right_index=True) #suffixes=["_all","_"+t])

        # merged[merged[("QApos_x","pred")] != merged[("QApos_y","pred")]].filter(regex="QApos")
    #     print(merged)
        print(t, t2)
        print("Correlation",merged[[("QApos_x","acc"),("QApos_y","acc")]].corr().iloc[0,1])
        print( pd.DataFrame(confusion_matrix(merged[("QApos_x","acc")], merged[("QApos_y","acc")], labels=[True,False]), [True,False],[True,False]))

# Convert To Conllu

In [3]:
import pandas as pd
df = pd.read_excel('results_comp_ench_4d_nonsoftmaxed_final_corrected.xlsx', header=[0,1])

In [101]:
df2 = df[("inputs","index")].str.split("-", expand=True)
df2.set_axis(axis=1, labels=[["myindex"]*4,["sid","aid","wid","mid"]], inplace=True)
df = df.join(df2)

In [234]:
df.inputs

,OOV,in_diac,in_letter,index,ar
0,0.0,aao,wEn,fourty-1103-1-1,وعن
5,0.0,a,>by,fourty-1103-2-1,أبي
10,NaN,aoaaa,vElbp,fourty-1103-3-1,ثعلبة
15,NaN,uai~i,Alx$ny,fourty-1103-4-1,الخشني
20,NaN,uoui,jrvwm,fourty-1103-5-1,جرثوم
25,0.0,i,bn,fourty-1103-6-1,بن
30,NaN,aoiK,nA$r,fourty-1103-7-1,ناشر
35,0.0,NaN,-,fourty-1103-8-1,۪
40,0.0,NaN,rDy,fourty-1103-9-1,رضي
45,0.0,~au,Allh,fourty-1103-10-1,الله


In [311]:
%load_ext autoreload
%autoreload 2
from buckwalter import bw2utf, utf2bw
bw2utf(",")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


','

In [312]:
def getFeats(results, mid):
    feats = [x.replace("QA","").capitalize()+
             "="+
             results[x][mid].capitalize() 
             for x in results 
             if x != "QApos" and
             mid < len(results[x]) and 
             x != "QAupos" and 
             results[x][mid]!='0']
    return "|".join(feats) or "_"


def getConllU(df):
    lines = []
    sentences = []
    sentences_chars = []
    words = []
    id = 1
    prev = [df.iloc[0].myindex.sid,df.iloc[0].myindex.aid]
    lines.append("# " +str(prev))
    for x in range(df.shape[0]):
        row = df.iloc[x]
        if [row.myindex.sid,row.myindex.aid] != prev:
            sentences.append(lines)
            sentences_chars.append(words)
            words = []
            lines = []
            id =1
            prev = [row.myindex.sid,row.myindex.aid]
            lines.append("# " +str(prev))

        ## segmentation
        segs = row.seg.pred.split("+")
        diacs = row.out_diac.pred.split("+")
        letter = row.out_letter.pred.split("+")
        chars = [int(s) for s in segs if s != '']
        maxseg = max(chars)
        forms = [[] for _ in range(maxseg+1)]
        for s,d,l in zip(chars,diacs,letter):
            if not s and s !=0:
                break
            forms[s].append(l)
            forms[s].append(d)
        forms = ["".join(f) for f in forms]

        results = {x: row.loc[(x,"pred")].split("/") for x in df.columns.levels[0] if x.startswith("QA") and type(row.loc[(x,"pred")]) == str}

        if len(forms) > 1:
            lines.append("\t".join([str(id)+"-"+str(id+len(forms)-1),bw2utf(row.inputs.in_letter), "_", "_", "_", "_", "_", "_", "_", "_"]))
        for mid, form in enumerate(forms):
            if form != "$":
                words.append(form)
                lines.append("\t".join([str(id+mid), 
                                        bw2utf(form) or "±", 
                                        "_",
                                        row.QAupos.pred.split("/")[mid] if mid < len(row.QAupos.pred.split("/")) else "_", 
                                        row.QApos.pred.split("/")[mid] if mid < len(row.QApos.pred.split("/")) else "_", 
                                        getFeats(results,mid), "_", "0", "_", "_", "_"]))
        id += len(forms)
    sentences.append(lines)
    sentences_chars.append(words)
    return sentences, sentences_chars

sentences_chars,chars = getConllU(df)

In [313]:
from conll18_ud_eval import load_conllu
gold = load_conllu(open("goldstandard.test.conllu"))

In [314]:
import edit_distance
import re

In [315]:
def removeDiac(x):
    return re.sub("[ًٌٍَُِّْ]","",x)

In [316]:
g = [removeDiac("".join(gold.characters[x.start:x.end])) for x in gold.sentences]

In [317]:
p = [removeDiac(bw2utf("".join(x))) for x in chars]

In [318]:
indexes = []
for x in g:
    mini = 10000
    for j,y in enumerate(p):
        dist = edit_distance.SequenceMatcher(x[:30],y[:30]).distance()
        if dist < mini:
            mini = dist
            best = j
            besty = y
    indexes.append(best)

        

In [319]:
len(set(indexes)), len(g), len(p)

(166, 168, 169)

In [320]:
# [(p[x],g[i]) for i, x in enumerate(indexes)]

In [321]:
# x = "\n\n".join(["\n".join(x) for x in sentences_chars])
# %store x > ensemble.test.conllu.original2

In [322]:
x = removeDiac("\n\n".join(["\n".join(sentences_chars[x]) for i, x in enumerate(indexes)]))

In [323]:
%store x > ensemble.test.conllu

Writing 'x' (str) to file 'ensemble.test.conllu'.
